In [4]:
import psycopg2

hostname = 'postgres'  
port = '5432'  
database = 'airbnb'  
username = 'postgres'  
password = 'postgres' 

def get_db():
    try:
        conn = psycopg2.connect(
            host=hostname,
            port=port,
            database=database,
            user=username,
            password=password
        )
        return conn
    except Exception as e:
        print(f'Error while connecting to db {e}')
        return None

In [7]:
import inspect, re

def varname(p):
  for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
    m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
    if m:
      return m.group(1)
    

def create_schema(db):
    dim_listing_sc = """
                CREATE TABLE IF NOT EXISTS dim_listing (
                    listing_id INT PRIMARY KEY,
                    url VARCHAR(255),
                    name VARCHAR(255),
                    description TEXT,
                    picture_url VARCHAR(255),
                    property_type VARCHAR(50),
                    room_type VARCHAR(50),
                    amenities TEXT[],
                    license VARCHAR(50),
                    instant_booking BOOLEAN
                );
        """
    dim_date_sc = """
                CREATE TABLE dim_date (
                    date_id INT PRIMARY KEY,
                    date DATE,
                    day INT,
                    month INT,
                    year INT
                );
            """
    dim_neighbourhood_sc = """
            CREATE TABLE dim_neighbourhood (
                neighbourhood_id INT PRIMARY KEY,
                obj JSON 
            );
            """
    dim_location_sc = """
            CREATE TABLE dim_location (
                location_id INT PRIMARY KEY,
                neighbourhood_id INT,
                latitude DECIMAL(9, 6),
                longitude DECIMAL(9, 6),
                geo_address VARCHAR(255),
                FOREIGN KEY (neighbourhood_id) REFERENCES dim_neighbourhood(neighbourhood_id)
            );
            """
    dim_host_sc = '''
        CREATE TABLE IF NOT EXISTS dim_host (
            host_id INT PRIMARY KEY,
            name VARCHAR(255),
            url VARCHAR(255),
            since_date_id INT,
            place VARCHAR(255),
            about TEXT,
            response_time VARCHAR(50),
            response_rate DECIMAL(5, 2),
            acceptance_rate DECIMAL(5, 2),
            is_superhost BOOLEAN,
            neighbourhood VARCHAR(255),
            verifications TEXT,
            total_listings INT,
            identity_verified BOOLEAN,
        FOREIGN KEY (since_date_id) REFERENCES dim_date(date_id)

        );
        '''
    dim_reviews_sc = """
        CREATE TABLE dim_reviews (
            review_id INT PRIMARY KEY,
            reviewer_name VARCHAR(255),
            reviewer_id INT,
            comments TEXT,
            date_id INT,
            listing_id INT,
            FOREIGN KEY (date_id) REFERENCES dim_date(date_id),
            FOREIGN KEY (listing_id) REFERENCES dim_listing(listing_id)
        );
        """
    fact_booking_sc = """
        CREATE TABLE fact_booking (
            booking_id INT PRIMARY KEY,
            host_id INT,
            listing_id INT,
            date_id INT,
            location_id INT,
            accommodates INT,
            bathrooms DECIMAL(3, 1),
            bedrooms INT,
            beds INT,
            price DECIMAL(10, 2),
            reviews_per_month DECIMAL(5, 2),
            FOREIGN KEY (host_id) REFERENCES dim_host(host_id),
            FOREIGN KEY (listing_id) REFERENCES dim_listing(listing_id),
            FOREIGN KEY (date_id) REFERENCES dim_date(date_id),
            FOREIGN KEY (location_id) REFERENCES dim_location(location_id)
        );
        """
    
    order = [dim_neighbourhood_sc,dim_date_sc,dim_listing_sc,dim_location_sc,dim_host_sc,dim_reviews_sc,fact_booking_sc]
    try:
        cursor = db.cursor()
        for q in order:
            cursor.execute(q)
            print(f'Loading {varname(q)}')
        db.commit()
        print(f"Added schema")
    except Exception as e:
        print(f'Exception while creating tables in db {e}')
        raise e 
db = get_db()
create_schema(db)

Loading q
Loading q
Loading q
Loading q
Loading q
Loading q
Loading q
Added schema


In [16]:
db.cursor().excecute()

AttributeError: 'psycopg2.extensions.cursor' object has no attribute 'excecute'